# 2. Microsoft Foundry Agent - 작업 단계별 표시

이 노트북에서는 **Microsoft Foundry의 Agent Service**를 사용하여 AI 에이전트의 작업 과정을 단계별로 표시하는 방법을 보여줍니다.

## 주요 내용
- 🤖 **Microsoft Foundry Agent**: 에이전트 생성 및 실행
- 📋 **Run Steps**: 에이전트 실행 단계 추적 API
- 🔧 **도구 호출 표시**: Code Interpreter, File Search, Function 등
- 📊 **실시간 상태**: 스트리밍 이벤트로 진행 상황 표시

> 📖 참고: [Microsoft Foundry Agent Service 문서](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/)

---

## 2.1 환경 설정 및 라이브러리 임포트

In [1]:
# 필요한 패키지 설치
%pip install --quiet azure-ai-projects azure-identity python-dotenv


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import time
import json
from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown, clear_output

# Azure AI SDK
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

from azure.ai.projects.models import PromptAgentDefinition
from azure.ai.projects.models import CodeInterpreterTool
from azure.ai.projects.models import WebSearchPreviewTool, ApproximateLocation

# 환경 변수 로드
load_dotenv()

print("✅ 라이브러리 로드 완료!")

✅ 라이브러리 로드 완료!


In [3]:
# Microsoft Foundry Project 클라이언트 설정
PROJECT_ENDPOINT = os.getenv("AZURE_AI_PROJECT_ENDPOINT", "")

if PROJECT_ENDPOINT:
    credential = DefaultAzureCredential()
    client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)
    print("✅ Microsoft Foundry Project 클라이언트 연결 완료!")
    print(f"   엔드포인트: {PROJECT_ENDPOINT}")
else:
    client = None
    print("⚠️ AZURE_AI_PROJECT_ENDPOINT가 설정되지 않았습니다.")
    print("   .env 파일에 엔드포인트를 추가해주세요.")
    print("   예: https://<project-name>.services.ai.azure.com/api/projects/<project-id>")

✅ Microsoft Foundry Project 클라이언트 연결 완료!
   엔드포인트: https://ms-foundry-1023.services.ai.azure.com/api/projects/ms-foundry


## 2.2 WebSearch 에이전트 생성 및 테스트

In [8]:
# 2.2 WebSearchAgent 생성
AGENT_NAME = "WebSearchAgent"
WEB_SEARCH_INSTRUCTIONS = """너는 Tools에 등록된 Web search 기반으로 답변하는 에이전트입니다.

중요 규칙:
1. 최신 정보가 필요한 질문에는 반드시 웹 검색을 사용하세요
2. 검색 결과를 기반으로 정확하고 최신의 정보를 제공하세요
3. 답변 시 출처 URL을 포함하세요
4. 여러 출처의 정보를 종합하여 균형잡힌 답변을 제공하세요
5. 검색 결과가 불충분하면 추가 검색을 수행하세요"""

try:
    definition = PromptAgentDefinition(
        model="gpt-4.1",
        instructions=WEB_SEARCH_INSTRUCTIONS,
        tools=[
          WebSearchPreviewTool()
        #   WebSearchPreviewTool(
        #     user_location=ApproximateLocation(country="GB", city="London", region="London")
        #   )
        ]
    )
    
    agent = client.agents.create(
        name=AGENT_NAME,
        definition=definition
    )
    
    print(f"✅ {AGENT_NAME} 생성 완료!")
    print(f"   ID: {agent.id}")
    print(f"   Name: {agent.name}")
    print(f"   Model: {definition.model}")
    print(f"   Tools: Web Search")
    
except Exception as e:
    print(f"⚠️ 에이전트 생성 실패: {e}")
    print("\n💡 해결 방법:")
    print("   1. 'gpt-4.1' 모델이 배포되었는지 확인")
    print("   2. Web Search가 프로젝트에서 활성화되었는지 확인")

⚠️ 에이전트 생성 실패: (conflict) Agent 'WebSearchAgent' already exists.
Code: conflict
Message: Agent 'WebSearchAgent' already exists.

💡 해결 방법:
   1. 'gpt-4.1' 모델이 배포되었는지 확인
   2. Web Search가 프로젝트에서 활성화되었는지 확인


### 함수 호출 테스트

In [9]:
try:
    print("⏳ WebSearchAgent 실행 중 (웹 검색 중...)")
    print(f"📍 Agent Name: {AGENT_NAME}")
    
    
    # SDK v2 - project client를 통해 OpenAI client 획듍
    openai_client = client.get_openai_client()
    
    # Conversation 생성
    conversation = openai_client.conversations.create()
    print(f"💬 Conversation ID: {conversation.id}")
    
    # Responses API 호출
    response = openai_client.responses.create(
        conversation=conversation.id,
        input="오늘 서울 날씨는 어떤가요?",
        extra_body={"agent": {"name": AGENT_NAME, "type": "agent_reference"}},
    )
    
    print("\n" + "=" * 80)
    print(f"🌐 {AGENT_NAME} 응답:")
    print("=" * 80)
    print(response.output_text)
    
    # Conversation 정리
    openai_client.conversations.delete(conversation_id=conversation.id)
    print("\n✅ 웹 검색 도구가 실시간 정보를 가져왔습니다!")
    
except NameError:
    print("⚠️ agent 또는 client가 정의되지 않았습니다.")
    print(f"💡 위의 환경 설정 및 {AGENT_NAME} 생성 셀을 먼저 실행하세요.")
    
except Exception as e:
    print(f"⚠️ 테스트 실패: {e}")
    import traceback
    print(f"\n상세 오류:\n{traceback.format_exc()}")

⏳ WebSearchAgent 실행 중 (웹 검색 중...)
📍 Agent Name: WebSearchAgent
💬 Conversation ID: conv_a75611f46127b94600o33CDmPZSkh7iyz9TshS29j7ypajXMAv

🌐 WebSearchAgent 응답:
2026년 2월 2일 서울의 날씨는 오전에는 최저기온이 -5도, 낮 최고기온은 1도 내외로, 영하권의 추운 날씨가 이어집니다. 전국적으로 눈 소식이 있으며, 일부 지역에는 대설특보가 발효 중이므로 빙판길과 결빙에 각별한 주의가 필요합니다. 낮에는 기온이 조금 오르겠지만, 전반적으로 춥고 눈이 내리는 겨울 날씨가 예상됩니다. 외출 시 따뜻하게 입고 미끄러운 도로 안전을 유의하세요.  
출처: [국민일보](https://www.kmib.co.kr/article/view.asp?arcid=0029335222), [조선일보](https://www.chosun.com/national/national_general/2026/02/02/T5ZE3UNZ6FBRTICJRMQAD4NGXU/), [weather.com](https://weather.com/ko-KR/weather/monthly/l/dcc67ddf015a546a905e781b75a39dd66381a1182af289ffe8c8b43ba0feafd6)

✅ 웹 검색 도구가 실시간 정보를 가져왔습니다!


### 스트리밍 방식 함수 호출 테스트

In [11]:
# Conversation 생성
conversation = openai_client.conversations.create()
print(f"💬 Conversation ID: {conversation.id}")
print()
    
stream_response = openai_client.responses.create(
    stream=True,
    conversation=conversation.id,
    input="오늘 서울 날씨는 어떤가요?",
    extra_body={"agent": {"name": AGENT_NAME, "type": "agent_reference"}},
)

print("🤖 응답: ", end="", flush=True)

for event in stream_response:
    if event.type == "response.created":
        pass  # 시작 시 별도 출력 불필요
    elif event.type == "response.output_text.delta":
        # 각 delta를 연결하여 출력 (줄바꿈 없이)
        print(event.delta, end="", flush=True)
    elif event.type == "response.output_item.done":
        if event.item.type == "message":
            item = event.item
            if item.content[-1].type == "output_text":
                text_content = item.content[-1]
                # URL 인용 출력
                if text_content.annotations:
                    print("\n\n📚 출처:")
                    for annotation in text_content.annotations:
                        if annotation.type == "url_citation":
                            print(f"   🔗 {annotation.url}")
    elif event.type == "response.completed":
        print("\n\n✅ 응답 완료!")

💬 Conversation ID: conv_b40486b9beca025e00SYRFW1jJ9X5aHOlXoXVwPbfhP5wXy1ba

🤖 응답: 오늘 서울은 영하로 매우 추운 날씨이며, 오전에는 맑다가 점차 구름이 많아질 전망입니다. 오전 최저기온은 약 -6도, 낮에는 최고 0도에서 1도까지 오를 것으로 보입니다. 밤부터는 눈이 내릴 가능성도 있어 귀가길에는 미끄럼 사고에 유의하셔야 합니다. 체감온도는 실제 기온보다 더 낮을 수 있으니, 외출 시 옷을 따뜻하게 챙기시기 바랍니다[이투데이 오늘날씨](https://www.etoday.co.kr/weather/today-weather?LV0=0&LV1=%EC%84%9C%EC%9A%B8), [Weather.com 서울 시간별예보](https://weather.com/ko-KR/weather/hourbyhour/l/82e46175f97c224acf6b95afc4934fbae0e4ba123adcee8a52b7be97c303467b), [뉴스1 오늘날씨](https://www.news1.kr/society/weather-disaster/6057697).

📚 출처:
   🔗 https://weather.com/ko-KR/weather/hourbyhour/l/82e46175f97c224acf6b95afc4934fbae0e4ba123adcee8a52b7be97c303467b
   🔗 https://www.etoday.co.kr/weather/today-weather?LV0=0&LV1=%EC%84%9C%EC%9A%B8
   🔗 https://www.news1.kr/society/weather-disaster/6057697


✅ 응답 완료!


## 2.3 진행 상황 표시 함수

에이전트 실행 단계를 시각적으로 표시하는 간단한 헬퍼 함수들입니다.

In [12]:
# 단계 유형별 아이콘
STEP_ICONS = {
    "thread_created": "📝",
    "message_added": "💬",
    "thinking": "🧠",
    "tool_calls": "🔧",
    "code_interpreter": "💻",
    "file_search": "📂",
    "function_call": "⚡",
    "message_creation": "✍️",
    "completed": "✅",
    "error": "❌",
}


def display_steps(steps: list, title: str = "Microsoft Foundry Agent 실행 중", is_complete: bool = False):
    """
    단계 목록을 HTML로 표시
    
    Args:
        steps: [{"type": str, "text": str, "status": str, "time": float}, ...]
        title: 표시할 제목
        is_complete: 완료 여부
    """
    if is_complete:
        bg_color = "rgba(76, 175, 80, 0.15)"
        border_color = "rgba(76, 175, 80, 0.4)"
        title_color = "#81c784"
    else:
        bg_color = "rgba(33, 150, 243, 0.15)"
        border_color = "rgba(33, 150, 243, 0.4)"
        title_color = "#64b5f6"
    
    html = f"""
    <div style="font-family: 'Segoe UI', sans-serif; padding: 15px; background: {bg_color}; 
                border: 1px solid {border_color}; border-radius: 10px; max-width: 650px;">
        <div style="font-weight: bold; color: {title_color}; margin-bottom: 12px; font-size: 1.1em;">
            🤖 {title}
        </div>
        <div style="border-left: 3px solid #0078d4; padding-left: 12px;">
    """
    
    for step in steps:
        step_type = step.get("type", "")
        icon = STEP_ICONS.get(step_type, "📌")
        status = step.get("status", "running")
        
        # 상태별 색상
        if status == "completed":
            status_icon = "✅"
            text_color = "#81c784"
        elif status == "running":
            status_icon = "⏳"
            text_color = "#64b5f6"
        elif status == "error":
            status_icon = "❌"
            text_color = "#ef5350"
        else:
            status_icon = "⚪"
            text_color = "#9e9e9e"
        
        time_str = f"+{step.get('time', 0):.1f}s" if step.get('time') else ""
        
        html += f"""
        <div style="padding: 6px 0; color: {text_color};">
            {status_icon} {icon} {step.get("text", "")}
            <span style="color: #bdbdbd; font-size: 0.85em; float: right;">{time_str}</span>
        </div>
        """
    
    html += "</div></div>"
    display(HTML(html))


def display_final_response(response_text: str, total_steps: int, total_time: float):
    """최종 응답을 표시"""
    html = f"""
    <div style="font-family: 'Segoe UI', sans-serif; padding: 15px; background: rgba(76, 175, 80, 0.15); 
                border: 1px solid rgba(76, 175, 80, 0.4); border-radius: 10px; max-width: 650px; margin-top: 10px;">
        <div style="font-weight: bold; color: #81c784; margin-bottom: 8px;">
            ✅ 작업 완료!
        </div>
        <div style="color: #e0e0e0; font-size: 0.9em;">
            📊 총 {total_steps}개 단계 | ⏱️ {total_time:.2f}초
        </div>
    </div>
    """
    display(HTML(html))
    display(Markdown(response_text))


print("✅ 진행 상황 표시 함수 정의 완료!")

✅ 진행 상황 표시 함수 정의 완료!


## 2.4 WebSearch Agent 실행 함수

앞서 생성한 WebSearchAgent를 활용하여 진행 상황을 표시하는 함수들입니다.

### Responses API 구조

```
openai_client.responses.create()
 └── Stream Events
      ├── response.created: 응답 시작
      ├── response.output_text.delta: 텍스트 토큰 스트리밍
      ├── response.output_item.done: 항목 완료 (annotations 포함)
      └── response.completed: 응답 완료
```

In [17]:
def run_websearch_agent(user_message: str, show_sources: bool = True):
    """
    WebSearchAgent를 실행하고 응답을 반환
    
    Args:
        user_message: 사용자 질문
        show_sources: 출처 URL 표시 여부
    
    Returns:
        응답 텍스트
    """
    if not client:
        print("⚠️ client 또는 agent가 설정되지 않았습니다.")
        return None
    
    steps = []
    start_time = time.time()
    
    def add_step(step_type: str, text: str, status: str = "running"):
        steps.append({
            "type": step_type,
            "text": text,
            "status": status,
            "time": time.time() - start_time
        })
        clear_output(wait=True)
        display_steps(steps, title="WebSearch Agent 실행 중")
    
    def complete_last_step(new_text: str = None):
        if steps:
            steps[-1]["status"] = "completed"
            if new_text:
                steps[-1]["text"] = new_text
            clear_output(wait=True)
            display_steps(steps, title="WebSearch Agent 실행 중")
    
    try:
        # 1. OpenAI Client 획득
        add_step("thinking", "OpenAI Client 준비 중...")
        openai_client = client.get_openai_client()
        complete_last_step("OpenAI Client 준비 완료")
        
        # 2. Conversation 생성
        add_step("thread_created", "Conversation 생성 중...")
        conversation = openai_client.conversations.create()
        complete_last_step(f"Conversation 생성 완료 (ID: {conversation.id[:12]}...)")
        
        # 3. 웹 검색 시작
        add_step("tool_calls", "🌐 웹 검색 중...")
        
        response = openai_client.responses.create(
            conversation=conversation.id,
            input=user_message,
            extra_body={"agent": {"name": AGENT_NAME, "type": "agent_reference"}},
        )
        complete_last_step("웹 검색 완료")
        
        # 4. 응답 생성 완료
        add_step("message_creation", "응답 생성 중...")
        complete_last_step("응답 생성 완료")
        
        add_step("completed", "작업 완료", status="completed")
        
        # 결과 표시
        total_time = time.time() - start_time
        clear_output(wait=True)
        display_steps(steps, title="WebSearch Agent 완료", is_complete=True)
        display_final_response(response.output_text, len(steps), total_time)
        
        # 출처 표시
        if show_sources:
            for item in response.output:
                if item.type == "message" and item.content:
                    for content in item.content:
                        if content.type == "output_text" and content.annotations:
                            print("\n📚 출처:")
                            for annotation in content.annotations:
                                if annotation.type == "url_citation":
                                    print(f"   🔗 {annotation.url}")
        
        # Conversation 정리
        openai_client.conversations.delete(conversation_id=conversation.id)
        
        return response.output_text
        
    except Exception as e:
        add_step("error", f"오류 발생: {str(e)}", status="error")
        raise


print("✅ run_websearch_agent() 함수 정의 완료!")
print("   사용법: run_websearch_agent('오늘 서울 날씨는?')")

✅ run_websearch_agent() 함수 정의 완료!
   사용법: run_websearch_agent('오늘 서울 날씨는?')


In [18]:
# 실행 예제
response = run_websearch_agent("최근 AI 에이전트 트렌드에 대해 알려주세요.")

최근 AI 에이전트(Agentic AI) 트렌드는 다음과 같이 요약할 수 있습니다:

### 1. 단순 챗봇에서 자율적 실행 주체로 진화
- 기존 AI는 질문에 답하거나 콘텐츠를 생성하는 보조 도구에 머물렀으나, 최근 AI 에이전트는 스스로 목표를 설정, 계획, 실행하며 일까지 처리하는 ‘일하는 AI’로 변화하고 있습니다. 반복적이고 소모적인 업무는 AI가 맡고, 인간은 판단과 전략에 집중할 수 있게 됩니다[네이버 블로그](https://blog.naver.com/dongkore/224167392481).

### 2. ‘에이전틱 워크플로(agentic workflow)’ 확산
- 단일 챗봇이 아닌, 여러 에이전트가 기업 전반에 걸쳐 복잡한 업무를 분담·협업하는 구조로 진화하고 있습니다. 2026년은 모든 기업/개인 업무에 AI 에이전트가 깊이 통합되는 전환점으로, 진짜 실무를 AI가 처리하는 시대가 열리고 있습니다[AI Times](https://www.aitimes.kr/news/articleView.html?idxno=37812), [StartupN](https://www.startupn.kr/news/articleView.html?idxno=56188).

### 3. 급격한 시장 성장 및 도입 확대
- 글로벌 조사기관은 2026년까지 조직의 82%가 AI 에이전트 도입을 계획 중이라고 밝힙니다. 시장 성장률은 연 40~50%에 달하며, AI 에이전트 분야의 스타트업 투자도 2024년 기준 3배 증가했습니다[KT Cloud Tech Story](https://tech.ktcloud.com/entry/2025-12-ktcloud-ai-agents-%EC%82%B0%EC%97%85-%ED%8C%A8%EB%9F%AC%EB%8B%A4%EC%9E%84).

### 4. 멀티 모델·실시간 처리·자율 협업
- 기업들은 단일 챗봇 형태에서 벗어나, 각 영역별로 여러 에이전트가 실시간으로 데이터를 처리하고, 서로 협업해 실제 운영을 맡는 멀티 에이전트 체계로 이동 중입니다. 예를 들어 AI가 고객 문의, 내부 회계, 보안 관리 등 다양한 팀 업무를 동시에 처리합니다[StartupN](https://www.startupn.kr/news/articleView.html?idxno=56188).

### 5. 빅테크와 오픈AI 등 대형 기업 주도
- 오픈AI, 구글, 마이크로소프트 등 빅테크가 범용 AI 에이전트 플랫폼을 앞장서서 선보이며, 오픈소스와 클로즈드소스 모델의 품질 격차도 줄어들고 있습니다. 모델 운영 비용이 빠르게 감소하며, 다양한 산업 분야에 적용이 확대됩니다[AIMatters](https://aimatters.co.kr/news-report/ai-report/20786).

### 6. 산업·업무별 활용 및 ROI 증가
- 금융, 제조, 헬스케어, 고객 서비스, IT운영, 개발 등 산업 전반에 도입되고 있습니다. 특히 헬스케어와 금융 분야에서 AI 에이전트가 높은 ROI와 업무 자동화 효과를 창출하고 있으며, 기업의 업무 재설계와 경쟁력 강화에 핵심 도구로 활용됩니다[KPMG 보고서](https://assets.kpmg.com/content/dam/kpmg/kr/pdf/2025/issue-monitor/%EC%82%BC%EC%A0%95KPMG-AI%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8-20250908.pdf).

### 7. 주요 기술 변화
- 대형 언어모델(LLM) 기반 추론, 자체 업무도구 연결, 데이터 큐레이션 및 실시간 실행 체계, 협업 AI(Agent2Agent) 표준 등이 주목받고 있습니다. 노코드/로우코드 에이전트 개발 플랫폼도 확대되고 있습니다[SK AX](https://www.skax.co.kr/insight/trend/3094).

### 시사점  
AI 에이전트는 더 이상 실험적인 차원이 아닌, 기업 및 일상 운영의 필수 주체로 자리잡아가는 중입니다. 성공적인 도입을 위해선 데이터 품질, AI 거버넌스, 보안과 책임 구조, 적합한 평가 체계 등의 기반을 갖추는 것이 중요합니다.

추가 참고:
- [CBInsights 요약](https://dpg.danawa.com/news/view?boardSeq=60&listSeq=5807860)
- [Google 2026 보고서 요약](https://www.dailybizon.com/news/articleView.html?idxno=59857)

더 상세한 산업별 적용 사례나 구체 기능, 실무 적용 전략이 필요하면 추가로 문의해 주세요.


📚 출처:
   🔗 https://tech.ktcloud.com/entry/2025-12-ktcloud-ai-agents-%EC%82%B0%EC%97%85-%ED%8C%A8%EB%9F%AC%EB%8B%A4%EC%9E%84
   🔗 https://aimatters.co.kr/news-report/ai-report/20786
   🔗 https://blog.naver.com/dongkore/224167392481
   🔗 https://www.startupn.kr/news/articleView.html?idxno=56188
   🔗 https://assets.kpmg.com/content/dam/kpmg/kr/pdf/2025/issue-monitor/%EC%82%BC%EC%A0%95KPMG-AI%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8-20250908.pdf
   🔗 https://dpg.danawa.com/news/view?boardSeq=60&listSeq=5807860
   🔗 https://www.aitimes.kr/news/articleView.html?idxno=37812
   🔗 https://www.dailybizon.com/news/articleView.html?idxno=59857
   🔗 https://www.skax.co.kr/insight/trend/3094


## 2.5 스트리밍 + 상세 단계 표시

`stream=True` 옵션으로 실시간 이벤트를 수신하며, 각 에이전트 작업 단계를 시각적으로 표시합니다.

- 참고
  - https://platform.openai.com/docs/api-reference/responses-streaming
  - https://learn.microsoft.com/ko-kr/azure/ai-foundry/agents/how-to/tools/web-search?view=foundry&pivots=python
  
### 웹 검색 에이전트 작업 단계

| 단계 | 이벤트 | 설명 |
|------|--------|------|
| 1️⃣ 준비 | - | OpenAI Client 및 Conversation 생성 |
| 2️⃣ 웹 검색 시작 | `response.created` | 에이전트가 웹 검색 도구 호출 |
| 3️⃣ 검색 결과 수신 | `response.output_item.done (function_call)` | 웹 검색 결과 반환 |
| 4️⃣ 응답 생성 | `response.output_text.delta` | 검색 결과 기반 응답 작성 |
| 5️⃣ 완료 | `response.completed` | 모든 작업 완료 |

In [ ]:
def run_websearch_agent_with_steps(user_message: str, show_sources: bool = True):
    """
    스트리밍 API로 WebSearchAgent 실행하며 각 단계를 상세히 표시
    
    Args:
        user_message: 사용자 질문
        show_sources: 출처 URL 표시 여부
    
    Returns:
        전체 응답 텍스트
    """
    if not client:
        print("⚠️ client 또는 agent가 설정되지 않았습니다.")
        return None
    
    steps = []
    start_time = time.time()
    full_response = ""
    sources = []
    web_search_started = False
    response_started = False
    
    def add_step(step_type: str, text: str, status: str = "running"):
        steps.append({
            "type": step_type,
            "text": text,
            "status": status,
            "time": time.time() - start_time
        })
        clear_output(wait=True)
        display_steps(steps, title="🌐 WebSearch Agent 실행 중")
    
    def complete_last_step(new_text: str = None):
        if steps:
            steps[-1]["status"] = "completed"
            if new_text:
                steps[-1]["text"] = new_text
            clear_output(wait=True)
            display_steps(steps, title="🌐 WebSearch Agent 실행 중")
    
    try:
        # 1. 준비 단계
        add_step("thinking", "OpenAI Client 준비 중...")
        openai_client = client.get_openai_client()
        complete_last_step("OpenAI Client 준비 완료")
        
        # 2. Conversation 생성
        add_step("thread_created", "Conversation 생성 중...")
        conversation = openai_client.conversations.create()
        complete_last_step(f"Conversation 생성 완료")
        
        # 3. 요청 전송
        add_step("message_added", f"질문 전송: '{user_message[:30]}...' " if len(user_message) > 30 else f"질문 전송: '{user_message}'")
        
        # 스트리밍 응답 요청
        stream_response = openai_client.responses.create(
            stream=True,
            conversation=conversation.id,
            input=user_message,
            extra_body={"agent": {"name": AGENT_NAME, "type": "agent_reference"}},
        )
        complete_last_step("질문 전송 완료")
        
        # 4. 스트리밍 이벤트 처리
        for event in stream_response:
            # 응답 시작 - 웹 검색 시작으로 간주
            if event.type == "response.created":
                if not web_search_started:
                    add_step("tool_calls", "🔍 웹 검색 시작...")
                    web_search_started = True
            
            # 웹 검색 함수 호출 감지
            elif event.type == "response.output_item.added":
                if hasattr(event, 'item'):
                    item_type = getattr(event.item, 'type', '')
                    if item_type == "function_call":
                        func_name = getattr(event.item, 'name', 'unknown')
                        if 'search' in func_name.lower() or 'web' in func_name.lower():
                            complete_last_step("🔍 웹 검색 시작")
                            add_step("tool_calls", "🌐 검색 쿼리 실행 중...")
            
            # 함수 호출 완료 (웹 검색 결과 수신)
            elif event.type == "response.output_item.done":
                if hasattr(event, 'item'):
                    item_type = getattr(event.item, 'type', '')
                    
                    # 함수 호출 완료 = 웹 검색 결과 수신
                    if item_type == "function_call":
                        complete_last_step("🌐 웹 검색 완료")
                        add_step("file_search", "📄 검색 결과 분석 중...")
                    
                    # 함수 호출 결과 처리
                    elif item_type == "function_call_output":
                        complete_last_step("📄 검색 결과 분석 완료")
                        add_step("message_creation", "✍️ 응답 생성 중...")
                    
                    # 메시지 완료 - URL 인용 수집
                    elif item_type == "message":
                        for content in event.item.content:
                            if content.type == "output_text" and content.annotations:
                                for annotation in content.annotations:
                                    if annotation.type == "url_citation":
                                        sources.append(annotation.url)
            
            # 텍스트 스트리밍 시작
            elif event.type == "response.output_text.delta":
                if not response_started:
                    # 웹 검색이 시작되지 않았다면 (직접 응답하는 경우)
                    if web_search_started and len(steps) > 0 and steps[-1]["type"] == "tool_calls":
                        complete_last_step("🌐 웹 검색 결과 수신 완료")
                        add_step("message_creation", "✍️ 응답 생성 중...")
                    elif not any(s["type"] == "message_creation" for s in steps):
                        add_step("message_creation", "✍️ 응답 생성 중...")
                    response_started = True
                
                full_response += event.delta
            
            # 응답 완료
            elif event.type == "response.completed":
                complete_last_step("✍️ 응답 생성 완료")
                add_step("completed", "✅ 모든 작업 완료!", status="completed")
        
        # 최종 결과 표시
        total_time = time.time() - start_time
        clear_output(wait=True)
        display_steps(steps, title="🌐 WebSearch Agent 완료", is_complete=True)
        display_final_response(full_response, len(steps), total_time)
        
        # 출처 표시
        if show_sources and sources:
            print("\n📚 참고한 출처:")
            for i, url in enumerate(sources, 1):
                print(f"   {i}. 🔗 {url}")
        
        # Conversation 정리
        openai_client.conversations.delete(conversation_id=conversation.id)
        
        return full_response
        
    except Exception as e:
        add_step("error", f"오류 발생: {str(e)}", status="error")
        raise


print("✅ run_websearch_agent_with_steps() 함수 정의 완료!")
print("   사용법: run_websearch_agent_with_steps('오늘 서울 날씨는?')")

In [ ]:
# 상세 단계별 스트리밍 실행 예제
response = run_websearch_agent_with_steps("2024년 가장 인기있는 프로그래밍 언어는?")

## 📋 요약

### 제공 함수

| 함수 | 설명 |
|------|------|
| `run_websearch_agent()` | 일반 호출 방식 (결과만 표시) |
| `run_websearch_agent_with_steps()` | 스트리밍 + 단계별 진행 상황 시각화 |

---

### 에이전트 작업 흐름

```
┌─────────────────────────────────────────────────────────┐
│  1️⃣  OpenAI Client 준비 완료                            │
│  2️⃣  Conversation 생성 완료                             │
│  3️⃣  질문 전송 완료                                     │
│  4️⃣  🔍 웹 검색 시작                                    │
│  5️⃣  🌐 웹 검색 완료                                    │
│  6️⃣  📄 검색 결과 분석 완료                             │
│  7️⃣  ✍️  응답 생성 완료                                 │
│  8️⃣  ✅ 모든 작업 완료!                                 │
└─────────────────────────────────────────────────────────┘
```

---

### 스트리밍 이벤트

| 이벤트 | 설명 |
|--------|------|
| `response.created` | 응답 시작 |
| `response.output_item.added` | 함수 호출 시작 (웹 검색 등) |
| `response.output_item.done` | 함수 호출 완료 / 메시지 완료 |
| `response.output_text.delta` | 응답 텍스트 토큰 스트리밍 |
| `response.completed` | 모든 작업 완료 |

---

### 핵심 API

```python
# 1. 클라이언트 설정
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)
openai_client = client.get_openai_client()

# 2. WebSearch 에이전트 생성
definition = PromptAgentDefinition(
    model="gpt-4.1",
    instructions="...",
    tools=[WebSearchPreviewTool()]
)
agent = client.agents.create(name="WebSearchAgent", definition=definition)

# 3. Conversation 생성
conversation = openai_client.conversations.create()

# 4. 스트리밍 호출
stream = openai_client.responses.create(
    stream=True,
    conversation=conversation.id,
    input="질문",
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}}
)

# 5. 이벤트 처리
for event in stream:
    match event.type:
        case "response.created":
            print("🔍 웹 검색 시작...")
        case "response.output_item.done":
            print("🌐 검색 완료")
        case "response.output_text.delta":
            print(event.delta, end="", flush=True)
        case "response.completed":
            print("✅ 완료!")
```

---

**다음 노트북** → Semantic Kernel 멀티 에이전트 워크플로우